# HW1 Part 2B

README:

Just run all the code cells. Sorry for that some codes may not fully show in the pdf due to the cell length. But you could COPY AND PASTE them somewhere to see the full codes.

In [1]:
# import necessary libraries
library(caret) # for train, predict, etc.
library(doParallel) # for parallel computing
library(klaR) # for "train"
library(quanteda) # for "textmodel_nb"
library(OpenImageR) # for "resizeImage"
library(h2o) # for "randomforest"

Loading required package: lattice
Loading required package: ggplot2
Loading required package: foreach
Loading required package: iterators
Loading required package: parallel
Loading required package: MASS
Package version: 1.3.4
Parallel computing: 2 of 8 threads used.
See https://quanteda.io for tutorials and examples.

Attaching package: ‘quanteda’

The following object is masked from ‘jupyter:irkernel’:

    View

The following object is masked from ‘package:utils’:

    View


----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


Attaching package: ‘h2o’

The following objects are masked from ‘package:stats’:

    cor, sd, var

The following objects are masked from ‘package:

In [2]:
# parallel computation
registerDoParallel(makeCluster(detectCores()))

In [3]:
# ALL INPUT HERE
dataTrain <- read.csv('train.csv')
dataTest <- read.csv('test.csv', header = F)
dataVal <- read.csv('val.csv')

In [4]:
# spilt features and labels in training data and validation data
train.feature <- dataTrain[,3:786]
train.label <- as.factor(dataTrain[,2])
val.feature <- dataVal[,2:785] 
val.label <- as.factor(dataVal[,1])

#### Stretched Bounding Function

In [5]:
bounding <- function(dataIN){
    cubic <- matrix(,nrow(dataIN),400) 
    for (i in 1:nrow(dataIN)){
        cubic28 <- matrix(dataIN[i, ],28,28)
        for (j in 1:28){
            if (Reduce("+",cubic28[j,])>0){
                lowerbound <- j
                break
            }
        }
        for (j in 28:1){
            if (Reduce("+",cubic28[j,])>0){
                upperbound <- j
                break
            }
        }
        for (j in 1:28){
            if (Reduce("+",cubic28[,j])>0){
                leftbound <- j
                break
            }
        }
        for (j in 28:1){
            if (Reduce("+",cubic28[,j])>0){
                rightbound <- j
                break
            }
        }
        cubic20 <- cubic28[lowerbound:upperbound, leftbound:rightbound]
        cubic20 <- apply(resizeImage(as.matrix.data.frame(cubic20), 20, 20, method = "bilinear"), 1, function(x)x*255)
        cubic[i, ] <- cubic20
    }
    return (cubic)
}

## Random Forest Model

In [6]:
h2o.init()


H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    /var/folders/zp/dfyztm4x1wlf6712j939s4qc0000gn/T//RtmpO10u1Y/h2o_dustin_started_from_r.out
    /var/folders/zp/dfyztm4x1wlf6712j939s4qc0000gn/T//RtmpO10u1Y/h2o_dustin_started_from_r.err


Starting H2O JVM and connecting: .. Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         3 seconds 392 milliseconds 
    H2O cluster timezone:       America/Chicago 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.20.0.2 
    H2O cluster version age:    2 months and 26 days  
    H2O cluster name:           H2O_started_from_R_dustin_xsr118 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   4.00 GB 
    H2O cluster total cores:    8 
    H2O cluster allowed cores:  8 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA

In [7]:
# random forest model function
RF = function(train, val, ntrees, depth, output){
    model = h2o.randomForest(y = "label", training_frame = as.h2o(train), validation_frame = as.h2o(val), ignore_const_cols = F, ntrees = ntrees, max_depth = depth, score_each_iteration = T) 
    if (output){
        print(h2o.hit_ratio_table(model, valid = T)[1,2])
    }
    return (model)
}

Untouched RF Models

In [8]:
# frame 1 & 2 are dataframes for untouched cases
frame1 = dataTrain[,2:786]
frame1$label <- as.factor(frame1$label)
frame2 = dataVal
frame2$label <- as.factor(frame2$label)

In [9]:
model5 = RF(frame1, frame2, 10, 4, T);
model7 = RF(frame1, frame2, 30, 4, T);
model9 = RF(frame1, frame2, 10, 16, T);
model11 = RF(frame1, frame2, 30, 16, T);

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
[1] 0.869
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
[1] 0.8745
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
[1] 0.975
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
[1] 0.978

Stretched Bounding RF Models

In [10]:
# frame 3 & 4 are dataframes for stretched bounding cases
frame3 <- as.h2o(list(frame1[,1], bounding(dataTrain[ ,3:786])))
frame4 <- as.h2o(list(frame2[,1], bounding(dataVal[ ,2:785])))
names(frame3)[1] = c("label")
names(frame4)[1] = c("label")

  |======================================================================| 100%
  |======================================================================| 100%


In [11]:
model6 = RF(frame3, frame4, 10, 4, T);
model8 = RF(frame3, frame4, 30, 4, T);
model10 = RF(frame3, frame4, 10, 16, T);
model12 = RF(frame3, frame4, 30, 16, T);

  |======================================================================| 100%
[1] 0.847
  |======================================================================| 100%
[1] 0.86
  |======================================================================| 100%
[1] 0.9735
  |======================================================================| 100%
[1] 0.981


### Use RF Model to Predict

In [12]:
colnames(dataTest) <- colnames(val.feature) # change Vi to Xi in column names
frameTest1 <- as.h2o(list(frame2[,1], dataTest))
names(frameTest1)[1] = c("label")

  |======================================================================| 100%


In [13]:
test.pred5 = h2o.predict(model5, as.h2o(frameTest1))
output = data.frame(ImageId = as.matrix(0:19999)[,1], Label = as.integer(as.matrix(test.pred5)[,1]))
write.csv(output, file = "chaox2_5.csv", row.names = F)

test.pred7 = h2o.predict(model7, as.h2o(frameTest1))
output = data.frame(ImageId = as.matrix(0:19999)[,1], Label = as.integer(as.matrix(test.pred7)[,1]))
write.csv(output, file = "chaox2_7.csv", row.names = F)

test.pred9 = h2o.predict(model9, as.h2o(frameTest1))
output = data.frame(ImageId = as.matrix(0:19999)[,1], Label = as.integer(as.matrix(test.pred9)[,1]))
write.csv(output, file = "chaox2_9.csv", row.names = F)

test.pred11 = h2o.predict(model11, as.h2o(frameTest1))
output = data.frame(ImageId = as.matrix(0:19999)[,1], Label = as.integer(as.matrix(test.pred11)[,1]))
write.csv(output, file = "chaox2_11.csv", row.names = F)

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


In [14]:
frameTest2 <- as.h2o(list(frame2[,1], bounding(dataTest)))
names(frameTest2)[1] = c("label")

  |======================================================================| 100%


In [20]:
test.pred6 = h2o.predict(model6, as.h2o(frameTest2))
output = data.frame(ImageId = as.matrix(0:19999)[,1], Label = as.integer(as.matrix(test.pred6)[,1]))
write.csv(output, file = "chaox2_6.csv", row.names = F)

test.pred8 = h2o.predict(model8, as.h2o(frameTest2))
output = data.frame(ImageId = as.matrix(0:19999)[,1], Label = as.integer(as.matrix(test.pred8)[,1]))
write.csv(output, file = "chaox2_8.csv", row.names = F)

test.pred10 = h2o.predict(model10, as.h2o(frameTest2))
output = data.frame(ImageId = as.matrix(0:19999)[,1], Label = as.integer(as.matrix(test.pred10)[,1]))
write.csv(output, file = "chaox2_10.csv", row.names = F)

test.pred12 = h2o.predict(model12, as.h2o(frameTest2))
output = data.frame(ImageId = as.matrix(0:19999)[,1], Label = as.integer(as.matrix(test.pred12)[,1]))
write.csv(output, file = "chaox2_12.csv", row.names = F)

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
